In [63]:
import time

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

N = 100
BITS = 2048
EXAMPLE_TEXT_128_bytes = """\
04cQbT3Il6ZxyJpO0QHoh6l7OE2RILAyTq7b2R0Cl0pUX4CgNv0nyd96wH5Rvp0yoYbb6Se8BZOcQKDRQVKhRe3LOG3VDHasECIYYYFs6BpMWDVjrdjswkjyDc8XVdbz\
"""

In [64]:
def generate_rsa_keys(bits):
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=bits
    )
    public_key = private_key.public_key()
    return private_key, public_key


def encrypt_message(message, public_key):
    encrypted = public_key.encrypt(
        message.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted


def decrypt_message(encrypted_message, private_key):
    decrypted = private_key.decrypt(
        encrypted_message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted.decode()


def measure_performance():
    print("Keys generating")
    start_time = time.time()
    keys = [generate_rsa_keys(BITS) for _ in range(N)]
    key_gen_time = time.time() - start_time
    
    print(f"Time to generate {N} keys: {key_gen_time:.3f} sec")
    print(f"Average time to generate 1 key: {key_gen_time / N:.3f} sec\n")

    print("Encryption")
    start_time = time.time()
    encrypted_messages = [encrypt_message(EXAMPLE_TEXT_128_bytes, public_key) for _, public_key in keys]
    encryption_time = time.time() - start_time
    
    print(f"Time to encrypt {N} messages: {encryption_time:.3f} sec")
    print(f"Average time to encrypt 1 message: {encryption_time / N:.5f} sec\n")

    print("Decryption")
    start_time = time.time()
    decrypted_messages = [decrypt_message(encrypted_message, private_key) 
                          for encrypted_message, (private_key, _) in zip(encrypted_messages, keys)]
    decryption_time = time.time() - start_time
    
    print(f"Time to decrypt {N} messages: {decryption_time:.3f} sec")
    print(f"Average time to decrypt 1 message: {decryption_time / N:.3f} sec\n")

    ans = list(set(decrypted_messages))
    if len(ans) == 1 and ans[0] == EXAMPLE_TEXT_128_bytes:
        print(f"All messages were decrypted correctly")
    else:
        print(f"Some messages weren't decrypted correctly")

In [65]:
measure_performance()

Keys generating
Time to generate 100 keys: 5.353 sec
Average time to generate 1 key: 0.054 sec

Encryption
Time to encrypt 100 messages: 0.006 sec
Average time to encrypt 1 message: 0.00006 sec

Decryption
Time to decrypt 100 messages: 0.143 sec
Average time to decrypt 1 message: 0.001 sec

All messages were decrypted correctly
